In [ ]:
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

### Executando data augmentation das colunas idoneidade_financeira, n_min_max_limitacao_atestados

In [ ]:
data_gold = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/dataset_bid_notices_weak_sup_hab_clean_selected.csv")
data_gold

In [ ]:
data_gold = data_gold.drop_duplicates()
data_gold

In [ ]:
def verificaLabels(data):
    for coluna in data.columns[1:]:
        print(data[coluna].value_counts())

In [ ]:
verificaLabels(data_gold)

In [ ]:
colunas = data_gold.columns[1:]
colunas

In [ ]:
def get_df_without_da(data, colunas, coluna_exception):
    dados_filtrados = []
    for indice in data.index:
        if (data.loc[indice, colunas[0]] == 1 or data.loc[indice, colunas[1]] == 1) and (data.loc[indice, coluna_exception] == 0):
            dados_filtrados.append(data.iloc[indice])
    return pd.DataFrame(dados_filtrados)

In [ ]:
verificaLabels(data_gold)

### Realizando o Data Augmentation

In [ ]:
data_augmentated = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/data_back_translated_n_min_max_idoneidade_financeira.csv')
data_augmentated.drop(columns=['Unnamed: 0'], inplace=True)
data_augmentated

In [ ]:
verificaLabels(data_augmentated)

In [ ]:
data_gold = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/dataset_bid_notices_weak_sup_hab_clean_selected.csv')
data_gold.drop_duplicates(inplace=True)
data_gold

In [ ]:
def get_df_with_da(data, colunas, coluna_exception):
    dados_filtrados = []
    for indice in data.index:
        if (data.loc[indice, colunas[0]] == 1 or data.loc[indice, colunas[1]] == 1) and (data.loc[indice, coluna_exception] == 0):
            dados_filtrados.append(data.iloc[indice])
    return pd.DataFrame(dados_filtrados)

In [ ]:
df_filtrado = get_df_without_da(data_gold, ['n_min_max_limitacao_atestados', 'idoneidade_financeira'], 'comprovante_localizacao')
df_filtrado

In [ ]:
verificaLabels(data_gold)

In [ ]:
def stratified_data_augmentation(real_data, data_augmentated, columns_data_augmented, column_exception, train_ratio = 0.8, test_ratio = 0.1, val_ratio = 0.1):
    # Remove as linhas em que a classe com data_augmentation é 1.
    data_without_data_augmented = real_data.drop(get_df_with_da(real_data, columns_data_augmented, column_exception).index)
    #data_without_data_augmented = real_data[real_data[column_data_augmented] != 1]
    mskf_train_test = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    mskf_test_val = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    # Pega as colunas que serão as classes, sem a coluna de texto.
    target=data_without_data_augmented.drop(['text'], axis=1).values
    # Divide em teste e treino 80/20 estratificado.
    train_index, test_index = list(mskf_train_test.split(X=data_without_data_augmented.values, y=target))[0]
    # Divisão em teste e validação 50/50 estratificado.
    target = data_without_data_augmented.iloc[test_index].drop(['text'], axis=1).values
    test_index, val_index = list(mskf_test_val.split(X=data_without_data_augmented.iloc[test_index].values, y=target))[0]
    
    # Datasets pré-criados.
    train_df = data_without_data_augmented.iloc[train_index]
    test_df = data_without_data_augmented.iloc[test_index]
    val_df = data_without_data_augmented.iloc[val_index]
    
    # Tratando o conjunto de dados com data_augmentation.
    data_real_column_data_augmented = get_df_with_da(real_data, columns_data_augmented, column_exception)
    #data_real_column_data_augmented = real_data[real_data[column_data_augmented] == 1]
    # Quantidade de amostras data augmentation e reais.
    len_samples = len(data_real_column_data_augmented) + len(data_augmentated)
    len_train, len_test, len_valid = round(len_samples * train_ratio), round(len_samples * test_ratio), round(len_samples * val_ratio)
    
    # Inserção de amostras de teste reais.
    if len_test <= len(data_real_column_data_augmented):
        samples_test = data_real_column_data_augmented.sample(len_test)
        data_real_column_data_augmented.drop(samples_test.index, inplace=True)
        test_df = pd.concat([test_df, samples_test]).reset_index(drop=True)
    else:
        raise ValueError('A quantidade de amostras de reais é insuficiente para a divisão de teste.')
    
    # Inserção de amostras de validação reais.
    if len_valid <= len(data_real_column_data_augmented):
        samples_valid = data_real_column_data_augmented.sample(len_valid)
        data_real_column_data_augmented.drop(samples_valid.index, inplace=True)
        val_df = pd.concat([val_df, samples_valid]).reset_index(drop=True)
    else:
        raise ValueError('A quantidade de amostras de reais é insuficiente para a divisão de validação.')
    
    # Inserção das amostras reais restantes e as de data_augmentation no conjunto de treino
    train_df = pd.concat([train_df, data_augmentated, data_real_column_data_augmented]).reset_index(drop=True)
    return train_df, test_df, val_df

In [ ]:
train_df, test_df, val_df = stratified_data_augmentation(data_gold, data_augmentated, ['n_min_max_limitacao_atestados', 'idoneidade_financeira'], 'comprovante_localizacao')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
val_df

In [ ]:
verificaLabels(train_df)

In [ ]:
verificaLabels(test_df)

In [ ]:
verificaLabels(val_df)

In [ ]:
train_df.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/bid_notices_weak_sup_hab_clean_train.csv', index=False)
test_df.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/bid_notices_weak_sup_hab_clean_test.csv', index=False)
val_df.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/bid_notices_weak_sup_hab_clean_val.csv', index=False)

In [ ]:
mskf_train_test = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
mskf_test_val = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=42)
target = data_gold.drop(['text'], axis=1).values
train_index, test_index = list(mskf_train_test.split(X=data_gold.values, y=target))[0]
target = data_gold.iloc[test_index].drop(['text'], axis=1).values
test_index, val_index = list(mskf_test_val.split(X=data_gold.iloc[test_index].values, y=target))[0]

df_train = data_gold.iloc[train_index].reset_index(drop=True)
df_test = data_gold.iloc[test_index].reset_index(drop=True)
df_val = data_gold.iloc[val_index].reset_index(drop=True)

In [ ]:
df_train

In [ ]:
df_val

In [ ]:
df_test

In [ ]:
df_train.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/Stratified/bid_notices_weak_sup_hab_clean_train.csv', index=False)
df_test.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/Stratified/bid_notices_weak_sup_hab_clean_test.csv', index=False)
df_val.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/Stratified/bid_notices_weak_sup_hab_clean_val.csv', index=False)